In [1]:
import os

In [2]:
%pwd

'd:\\GoFreeLabTechnologies\\Internship projects\\CI_CD_Pipeline\\CI_CD_Pipeline-ML_flow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\GoFreeLabTechnologies\\Internship projects\\CI_CD_Pipeline\\CI_CD_Pipeline-ML_flow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    load_data: Path
    save_model: Path
    params: dict


In [6]:
from ImageForgeryDetection.constants import *
from ImageForgeryDetection.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config = self.config.model_trainer
        params = self.params.trainer

        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            load_data=Path(config.load_data),
            save_model=Path(config.save_model),
            params=params
        )

        return model_trainer_config

In [ ]:
import joblib
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.metrics import Precision, F1Score, Recall
from ImageForgeryDetection import logger
import gc

c:\Users\hp\anaconda3\envs\cicdproject\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\hp\anaconda3\envs\cicdproject\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [9]:
class ModelTrainer(Sequence):
    def __init__(self, config: ModelTrainerConfig, **kwargs):
        """
        Initializes the ModelTrainer with configuration.
        """
        super().__init__(**kwargs)  # Initialize Sequence base class
        logger.info("Initializing ModelTrainer")
        self.config = config
        self.params = config.params
        self.model = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.train_indexes = None
        self.val_indexes = None

    def __len__(self):
        """
        Returns the number of training batches per epoch.
        """
        return int(np.floor(len(self.X_train) / self.params['batch_size']))

    def __getitem__(self, index):
        """
        Retrieves a training batch of data.
        """
        indexes = self.train_indexes[index * self.params['batch_size']:(index + 1) * self.params['batch_size']]
        X = [self.X_train[k] for k in indexes]
        y = [self.y_train[k] for k in indexes]
        return np.array(X), np.array(y)

    def on_epoch_end(self):
        """
        Shuffles training indices at the end of each epoch.
        """
        self.train_indexes = np.arange(len(self.X_train))
        np.random.shuffle(self.train_indexes)

    def get_validation_data(self):
        """
        Returns the validation data generator.
        """
        class ValidationGenerator(Sequence):
            def __init__(self, X, y, batch_size, **kwargs):
                super().__init__(**kwargs)  # Initialize Sequence base class
                self.X = X
                self.y = y
                self.batch_size = batch_size
                self.indexes = np.arange(len(self.X))

            def __len__(self):
                return int(np.floor(len(self.X) / self.batch_size))

            def __getitem__(self, index):
                indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
                X = [self.X[k] for k in indexes]
                y = [self.y[k] for k in indexes]
                return np.array(X), np.array(y)

        return ValidationGenerator(self.X_test, self.y_test, self.params['batch_size'])

    def load_data(self):
        """
        Loads X and y data from joblib files specified in config.
        """
        logger.info(f"Loading data from {self.config.load_data}")
        try:
            x_path = self.config.load_data / 'X_90.joblib'
            y_path = self.config.load_data / 'y.joblib'
            X = joblib.load(x_path)
            y = joblib.load(y_path)
            logger.info(f"Loaded X with shape {X.shape} and y with shape {y.shape}")
            return X, y
        except Exception as e:
            logger.error(f"Error loading data: {e}")
            raise

    def split_data(self, X, y):
        """
        Splits data into training and testing sets.
        """
        logger.info("Splitting data into train and test sets")
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42, stratify=y
            )
            logger.info(f"Train set: X_train shape {X_train.shape}, y_train shape {y_train.shape}")
            logger.info(f"Test set: X_test shape {X_test.shape}, y_test shape {y_test.shape}")
            return X_train, X_test, y_train, y_test
        except Exception as e:
            logger.error(f"Error splitting data: {e}")
            raise

    def preprocess_data(self, X_train, X_test, y_train, y_test):
        """
        Reshapes data for CNN input and initializes indices.
        """
        logger.info("Preprocessing data")
        try:
            # Reshape for CNN input (128x128x3)
            X_train = X_train.reshape(X_train.shape[0], 128, 128, 3)
            X_test = X_test.reshape(X_test.shape[0], 128, 128, 3)
            y_train = y_train.reshape(y_train.shape[0], 2)
            y_test = y_test.reshape(y_test.shape[0], 2)
            logger.info(f"Reshaped X_train to {X_train.shape}, X_test to {X_test.shape}")
            logger.info(f"Reshaped y_train to {y_train.shape}, y_test to {y_test.shape}")

            # Store reshaped data and initialize indices
            self.X_train, self.X_test = X_train, X_test
            self.y_train, self.y_test = y_train, y_test
            self.train_indexes = np.arange(len(self.X_train))
            np.random.shuffle(self.train_indexes)
            logger.info("Data preprocessing completed")
        except Exception as e:
            logger.error(f"Error preprocessing data: {e}")
            raise

    def build_model(self):
        """
        Builds and returns a CNN model.
        """
        logger.info("Building CNN model")
        try:
            model = tf.keras.Sequential([
                tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                tf.keras.layers.MaxPooling2D((2, 2)),
                tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(2, activation='softmax')
            ])
            logger.info("Model built successfully")
            return model
        except Exception as e:
            logger.error(f"Error building model: {e}")
            raise

    def train(self):
        """
        Trains the model with the specified configuration and saves the best model.
        """
        logger.info("Starting model training")
        try:
            # Build model
            self.model = self.build_model()
            
            # Compile model with parameters from params.yaml
            optimizer = self.params['optimizer']
            metrics = [metric.lower() if metric == 'accuracy' else getattr(tf.keras.metrics, metric)() 
                       for metric in self.params['metrics']]
            self.model.compile(optimizer=optimizer, 
                              loss='categorical_crossentropy', 
                              metrics=metrics)
            self.model.summary()

            # Define callbacks
            cal1 = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', 
                patience=self.params['patience'], 
                restore_best_weights=True
            )
            cal2 = tf.keras.callbacks.ModelCheckpoint(
                str(self.config.save_model / 'model.keras'), 
                monitor='val_loss', 
                save_best_only=True
            )
            
            # Train model with default Keras progress bar
            history = self.model.fit(
                self,
                epochs=self.params['epochs'],
                validation_data=self.get_validation_data(),
                callbacks=[cal1, cal2],
                verbose=1  # Use default Keras progress bar
            )
            logger.info("Model training completed")
            return history.history
        except Exception as e:
            logger.error(f"Error during training: {e}")
            raise

    def run_training_pipeline(self):
        """
        Runs the complete training pipeline: load, split, preprocess, and train.
        """
        logger.info("Starting training pipeline")
        try:
            # Load data
            X, y = self.load_data()
            
            # Split data
            X_train, X_test, y_train, y_test = self.split_data(X, y)
            
            # Free memory
            del X
            del y
            gc.collect()
            logger.info("Memory cleared after loading and splitting")
            
            # Preprocess data
            self.preprocess_data(X_train, X_test, y_train, y_test)
            
            # Train model
            history = self.train()
            logger.info("Training pipeline completed")
            return history
        except Exception as e:
            logger.error(f"Error in training pipeline: {e}")
            raise

In [ ]:
try:
        config_manager = ConfigurationManager()
        model_trainer_config = config_manager.get_model_trainer_config()
        trainer = ModelTrainer(config=model_trainer_config)
        history = trainer.run_training_pipeline()
except Exception as e:
        logger.error(f"Pipeline failed: {e}")
        raise

[2025-08-08 13:53:45,533: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-08 13:53:45,541: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-08 13:53:45,545: INFO: common: created directory at: artifacts]
[2025-08-08 13:53:45,547: INFO: common: created directory at: artifacts/model_trainer]
[2025-08-08 13:53:45,548: INFO: 1673120850: Initializing ModelTrainer]
[2025-08-08 13:53:45,551: INFO: 1673120850: Starting training pipeline]
[2025-08-08 13:53:45,553: INFO: 1673120850: Loading data from artifacts\data_preprocessing\pickle]


[2025-08-08 13:53:58,415: INFO: 1673120850: Loaded X with shape (9501, 49152) and y with shape (9501, 2)]
[2025-08-08 13:53:58,415: INFO: 1673120850: Splitting data into train and test sets]
[2025-08-08 13:54:02,002: INFO: 1673120850: Train set: X_train shape (7600, 49152), y_train shape (7600, 2)]
[2025-08-08 13:54:02,013: INFO: 1673120850: Test set: X_test shape (1901, 49152), y_test shape (1901, 2)]
[2025-08-08 13:54:02,868: INFO: 1673120850: Memory cleared after loading and splitting]
[2025-08-08 13:54:02,868: INFO: 1673120850: Preprocessing data]
[2025-08-08 13:54:02,868: INFO: 1673120850: Reshaped X_train to (7600, 128, 128, 3), X_test to (1901, 128, 128, 3)]
[2025-08-08 13:54:02,868: INFO: 1673120850: Reshaped y_train to (7600, 2), y_test to (1901, 2)]
[2025-08-08 13:54:02,965: INFO: 1673120850: Data preprocessing completed]
[2025-08-08 13:54:02,965: INFO: 1673120850: Starting model training]
[2025-08-08 13:54:02,965: INFO: 1673120850: Building CNN model]


c:\Users\hp\anaconda3\envs\cicdproject\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[2025-08-08 13:54:03,212: INFO: 1673120850: Model built successfully]


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     3,211,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,267,778 (12.47 MB)

 Trainable params: 3,267,778 (12.47 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
237/237 ━━━━━━━━━━━━━━━━━━━━ 52s 205ms/step - accuracy: 0.8290 - f1_score: 0.7199 - loss: 0.3349 - precision: 0.8290 - recall: 0.8290 - val_accuracy: 0.8660 - val_f1_score: 0.8256 - val_loss: 0.3202 - val_precision: 0.8660 - val_recall: 0.8660
Epoch 2/15
237/237 ━━━━━━━━━━━━━━━━━━━━ 49s 206ms/step - accuracy: 0.8605 - f1_score: 0.8071 - loss: 0.2843 - precision: 0.8605 - recall: 0.8605 - val_accuracy: 0.8782 - val_f1_score: 0.8364 - val_loss: 0.2741 - val_precision: 0.8782 - val_recall: 0.8782
Epoch 3/15
237/237 ━━━━━━━━━━━━━━━━━━━━ 47s 200ms/step - accuracy: 0.8797 - f1_score: 0.8369 - loss: 0.2628 - precision: 0.8797 - recall: 0.8797 - val_accuracy: 0.8803 - val_f1_score: 0.8432 - val_loss: 0.2544 - val_precision: 0.8803 - val_recall: 0.8803
Epoch 4/15
237/237 ━━━━━━━━━━━━━━━━━━━━ 47s 198ms/step - accuracy: 0.8882 - f1_score: 0.8477 - loss: 0.2352 - precision: 0.8882 - recall: 0.8882 - val_accuracy: 0.8771 - val_f1_score: 0.8399 - val_loss: 0.2601 - val_precision: 0.8771 -